In [ ]:
import os
import time
import cv2
import numpy as np
import onnxruntime as ort
from buildhat import Motor

# Initialize motors
motor_C = Motor('C')  # Right motor
motor_D = Motor('D')  # Left motor

# Motor speed settings
MOVE_SPEED = 75
TURN_SPEED = 65
CONF_THRESHOLD = 0.5

# Load the ONNX model
ort_session = ort.InferenceSession("/home/aman1234/Downloads/best.onnx")

# Initialize camera
cap = cv2.VideoCapture(0)  # USB cam or Arducam in /dev/video0

def preprocess(frame):
    img = cv2.resize(frame, (640, 640))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))  # CHW
    img = np.expand_dims(img, axis=0)   # Add batch dim
    return img

def move_towards(x_center):
    if x_center < 213:  # Left third
        print("Turning left")
        motor_C.start(TURN_SPEED)
        motor_D.start(-TURN_SPEED)
    elif x_center > 426:  # Right third
        print("Turning right")
        motor_C.start(-TURN_SPEED)
        motor_D.start(TURN_SPEED)
    else:  # Centered
        print("Moving forward")
        motor_C.start(MOVE_SPEED)
        motor_D.start(MOVE_SPEED)

def stop_all():
    motor_C.stop()
    motor_D.stop()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        input_tensor = preprocess(frame)
        input_name = ort_session.get_inputs()[0].name
        outputs = ort_session.run(None, {input_name: input_tensor})
        detections = outputs[0][0]  # Shape: (5, 8400)

        max_conf = 0
        best_x_center = None

        for det in detections.T:  # (5, 8400) → loop over 8400 detections
            x_center, y_center, w, h, conf = det
            if conf > CONF_THRESHOLD and conf > max_conf:
                max_conf = conf
                best_x_center = x_center * 640  # rescale from 0–1 to pixel

        if best_x_center:
            move_towards(best_x_center)
        else:
            print("No trash detected")
            stop_all()

        time.sleep(0.1)

except KeyboardInterrupt:
    print("Exiting...")
    stop_all()
    cap.release()
    cv2.destroyAllWindows()
